In [21]:
import sys
import os
import csv

import orjson
import ijson
import json
import torch
import numpy as np
import pandas

from typing import List, Optional
from tqdm import tqdm

sys.path.append(os.path.abspath("..")) # append calibration folder to path for McEval import

In [ ]:
data_path = "output/Qwen3-Coder-30B-A3B/Scenario.codegeneration_10_0.2_eval_all.json"
out_path = "output/Qwen3-Coder-30B-A3B/Scenario.codegeneration_10_0.2_eval_all.jsonl"
jsonlines = "/data/tyler/dok/viola/exchange/robin/codegeneration_10_0.2_eval_all.jsonl"
evaluation_data_path = "output/Qwen3-Coder-30B-A3B/preprocessed/codegeneration_10_0.2.jsonl"
topic_lookup_path = "/data/tyler/dok/viola/exchange/robin/topic_lookup.json"

import requests

def get_leetcode_questions():
    questions = []
    with open("/data/stud/klopp001/hiwi/logprob_project/LeetCode_Questions.tsv", newline='', encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t")
        for row in reader:
            data = {k: row[k] for k in ["Problem Name", "Topics"] if k in row}
            questions.append(data)
        
    for q in questions:
        if "Problem Name" in q:
            q["question_name"] = q.pop("Problem Name")
        if "Topics" in q:
            q["tags"] = q.pop("Topics")
        name = str((q["question_name"])).lower().strip()
        q["question_name"] = name.replace(" ", "-")
        tag_string = ""
        tag_string = str(q["tags"]).lower()
        q["tags"] = []
        current_tag = ""

        for i in range(0,len(tag_string)):
            if (tag_string[i]==" " and tag_string[i-1]==",") and len(current_tag)>2:
                q["tags"].append(current_tag[0:-1])
                current_tag = ""
            elif i==len(tag_string)-1:
                current_tag += tag_string[i]
                q["tags"].append(current_tag)
            else:
                current_tag += tag_string[i]

    return questions


def get_atcoder_questions():
    tag_list={}
    questions = []
    question_list= []

    keymap={'easy':"tag_search/Easy", 'ad-hoc':"tag_search/Ad-Hoc",
            'search':"tag_search/Searching", 'breadth-first search':"tags/Searching/BFS",
            'depth-first search':"tags/Searching/DFS", 'binary search':"tags/Searching/Binary-Search",
            'brute force':"tags/Searcging/BruteForce",'greedy':"tag_search/Greedy-Methods",
            'string':"tag_search/String", 'suffix array': "tags/String/Suffix-Array",
            'trie':"tags/String/Trie", 'rolling hash':"tags/String/Rolling-Hash",
            'math':"tag_search/Mathematics", 'bit manipulation':"tags/Mathematics/XOR", 
            'matrix':"tags/Mathematics/Matrix", 'probability and statistics':"tags/Mathematics/Probability",
            'combinatorics':"tags/Mathematics/Combinatorics", 'enumeration':"tags/Mathematics/Enumerate",
            'technique':"tag_search/Technique", 'sorting':"tags/Technique/Sort",
            'simulation':"tags/Technique/Simulation", 'recursion':"tags/Technique/Recursion",
            'divide and conquer':"tags/Technique/Divide-And-Conquer", 'two pointers': "tags/Technique/Two-Pointers",
            'constructive algorithms':"tag_search/Construct", 'graph':"tag_search/Graph",
            'shortest path': "tags/Graph/Shortest-Path", 'minimum spanning tree':"tags/Graph/Minimum-Spanning-Tree",
            'topological sort':"tags/Graph/Topological-Search", 'dynamic programming': "tag_search/Dynamic-Programming",
            'data structures':"tag_search/Data-Structure", 'stack':"tags/Data-Structure/stack",
            'queue':"tags/Data-Structure/Queue", 'union find':"tags/Data-Structure/Union-Find-Tree",
            'binary tree':"tags/Data-Structure/BIT", 'game':"tag_search/Game", 'backtracking':"tags/Game/Backtrack",
            'flow algorithm':"tag_search/Flow-Algorithms",'geometry':"tag_search/Geometry",
            'interactive':"tag_search/Interactive",'april fool':"tag_search/April-Fool",
            'marathon':"tag_search/Marathon"}
    for category, link in keymap.items():
        base_url= "https://atcoder-tags.herokuapp.com/"
        url = f"{base_url}{link}"
        tables = pandas.read_html(url)
        df = tables[0]
        reduced = df[["ID", "Name"]].copy()
        for c in reduced.columns:
                reduced[c] = reduced[c].where(reduced[c].isna(), reduced[c].astype(str).str.strip())
        reduced = reduced.replace(r"^\s*$", pandas.NA, regex=True)
        reduced = reduced.dropna(subset=["ID", "Name"])
        rows = reduced.to_dict(orient="records") 
        tag_list[category] = rows
    for tag in tag_list:
        for question_by_tag in tag_list[tag]:
                if question_by_tag["ID"] not in question_list:
                    data = {
                        "question_id": question_by_tag["ID"],
                        "question_name": question_by_tag["Name"],
                        "tags": [tag]
                    }
                    questions.append(data)
                    question_list.append(question_by_tag["ID"])
                else:  
                    for question in questions:
                        if question_by_tag["ID"]==question["question_id"]:
                            question["tags"].append(tag)

    return questions

 

def get_codeforces_questions():
    tag_map= {'dp':'dynamic programming', 'string':'strings', 'graphs':'graph', 'sortings':'sorting', 'dsu':'union find'}
    contests=[1873,1883,1899]
    api_url = "https://codeforces.com/api/problemset.problems"
    r = requests.get(api_url, timeout=20)
    r.raise_for_status()
    data = r.json()
    df = data["result"]
    problems = df["problems"]
    filtered = [row for row in problems if row.get("contestId") in contests]
    
    for row in filtered:
        if "index" in row:
            row["question_id"] = row.pop("index")
        if "contestId" in row:
            row["contest_id"] = row.pop("contestId")
        row["contest_id"]= str(row["contest_id"])
        index = (row["question_id"]).upper()
        row["question_id"]= f'{row["contest_id"]}_{index}'
        row["tags"] = [tag.lower() for tag in row["tags"]]
        row["tags"] = [tag_map.get(t,t) for t in row.get("tags", []) if isinstance(t, str)]

    return filtered
  


def create_question_file(leetcode,atcoder,codeforce):
    questions = {
        "leetcode": leetcode,
        "atcode": atcoder,
        "codeforces": codeforces
    }

    
    with open(topic_lookup_path, "w", encoding="utf-8") as f:
        json.dump(questions, f, ensure_ascii=False, indent=2)


leetcode = get_leetcode_questions()
atcoder = get_atcoder_questions()
codeforces = get_codeforces_questions()
create_question_file(leetcode,atcoder,codeforces)



In [ ]:


# convert to jsonl
def clean_logprobs(logprobs_list):
    for sequence in logprobs_list:
        for token in sequence:
            for token_id in token:
                token[token_id]['logprob'] = str(token[token_id]['logprob'])
    return logprobs_list

def convert_to_jsonl(data_path, out_path):
    with open(data_path, "rb") as infile, open(out_path, "wb") as outfile:
        for item in ijson.items(infile, "item"):  # iterate over each object in array
            item['pass@1'] =str(item['pass@1'])
            item['logprobs_list'] = clean_logprobs(item['logprobs_list'])
            line = orjson.dumps(item) + b""
            outfile.write(line)
            
# convert_to_jsonl(data_path, out_path)

    


In [ ]:
# Katarinas changes:

def top20_logprob_sum_f(sample: dict):
    top20 = []
    for sequence in sample["logprobs_list"]:
        top20seq = []
        count = 0                               #count = Token-Nr. (beginnend mit 0)
        for token in sequence:
            top20seq.append(0)
            for k in token.values():
                if (k["rank"]>= 1) and (k["rank"]<= 20):
                    top20seq[count]+= float (k["logprob"])
            count += 1
        top20.append(top20seq)
    return top20


def search(sample:dict):
    # This is tokenizer-dependent
    # Relevant tokens for Qwen3: 73594='```', 
    #                            12669='python', 
    #                            13874='``', 
    #                            19324='` '
    #                            41233='Ġ```Ċ'
     
    coding_tok_list = []
    for sequence in sample["logprobs_list"]:
        token_list = []

        index = 0
        for token in sequence:
            token_list.append(0)
            for id, k in token.items():
                if k["rank"]==1:
                    token_list[index]= id
            index += 1

        position = []
        for i in range(0,len(token_list)-2):
            if token_list[i]=='73594' and token_list[i+1]=='12669':
                if token_list[i+2]=='198':
                    position.append(i+3)
                else:
                    position.append(i+2)

        if len(position) > 0:
            position=position[-1:] # check this -> needs to handle incomplete last sequences
            
            for j in range(position[0],len(token_list)-1):
                if token_list[j]=='73594':
                    position.append(j-1)
                elif token_list[j]=='13874' and token_list[j+1]=='19324':
                    position.append(j-1)
                    
        
        if len(position)>2:
            position = [position[0]] # Ignore end position if multiple are found. @Katharina: -> why? 

        coding_tok_list.append(position)

    return coding_tok_list

def coding_logprob_f(position_list, logprob_list):
    coding_logprobs = {}
    first_token = []
    first_20_token = []
    last_token = []
    last_20_token= []

    #if len(position_list)==len(logprob_list): #
    assert len(position_list)==len(logprob_list), 'Position list and logprob list need to be of equal size'
    for seq in range(0,len(logprob_list)):
        seq_logprobs = logprob_list[seq]
        seq_positions = position_list[seq]

        if len(seq_positions)>=1 and len(seq_positions)<3:
            first_token.append(seq_logprobs[(seq_positions[0])])

            if (seq_positions[0]+20)<=(len(seq_logprobs)-1):
                first_20_token.append(float (np.mean(seq_logprobs[(seq_positions[0]):(seq_positions[0]+20)])))
            else:
                first_20_token.append([])
            
            if len(seq_positions)==2:
                last_token.append(seq_logprobs[(seq_positions[1])])
                last_20_token.append(float (np.mean(seq_logprobs[(seq_positions[1]-19):(seq_positions[1]+1)])))
            else:
                last_20_token.append([])
                last_token.append([])


        else:
            first_token.append([])
            first_20_token.append([])
            last_20_token.append([])
            last_token.append([])

    coding_logprobs = {
        "first_token": first_token,
        "first_20_token": first_20_token,
        "last_token": last_token,
        "last_20_token": last_20_token
    }

    
    return coding_logprobs


def test(sample):
    candidates = ('73594', '12669', '13874', '19324', '41233')

    hit = []
    for sequence in sample["logprobs_list"]:
        hit_seq = []
        index = 0                               #count = Token-Nr. (beginnend mit 0)
        for token in sequence:
            for id, k in token.items():
                if (k["rank"]==1 
                    and (k["decoded_token"].find("``")!= -1) 
                    and id not in candidates) :
                    hit_seq.append([id,k["decoded_token"], index])
                    
            index += 1
        if len(hit_seq)>0:
            hit.extend(hit_seq)
    if len(hit) > 0:
        print(f"Check sample {sample['question_title']}: {hit}")

def get_questions():
    with open(topic_lookup_path, "r", encoding="utf-8") as f:
        return json.load(f)



def get_topics(sample):
    topics = []

    platform = sample['platform']

    questions = get_questions()
    set = questions[platform]

    for question in set:
        if platform=='leetcode':
            if question["question_name"]==sample["question_title"]:
                topics = question["tags"]
            
            
        else:
            if question["question_id"]==sample['question_id']:
                topics = question["tags"]
                
    return topics





        

def prepare_sample(sample: dict):
    logprobs = [
                [
                    k["logprob"] for d in sequence for k in filter(lambda x: x["rank"] == 1, d.values())
                ] for sequence in sample["logprobs_list"]
            ]
    
    topics = get_topics(sample)
    
    code_token_idxs = [extract_code_tokens(seq) for seq in sample["logprobs_list"]]

    
    logprob_sum = top20_logprob_sum_f(sample)
    coding_tok_pos = search(sample)
    coding_log_probs = coding_logprob_f(code_token_idxs,logprob_sum)

    #test(sample)


    return {
        "id": sample["question_id"],
        "name": sample["question_title"],
        "topics": topics,
        "prompt": sample["question_content"],
        "program": sample["code_list"],
        "language": 'python',
        "output_size": [len(output) for output in sample["output_list"]],
        "token_count": [len(logprobs) for logprobs in sample["logprobs_list"]],
        "is_correct": sample["graded_list"],
        "token_logprobs": logprobs,
        'cumulative_logprob': [sum([float(x) for x in l]) for l in logprobs],
        "top_logprobs_first_token": [sequence[0] for sequence in logprob_sum],
        "top_logprobs_first_20_tokens": [(float (np.mean(sequence[0:20]))) for sequence in logprob_sum],
        "top_logprobs_last_token": [sequence[-1] for sequence in logprob_sum],
        "top_logprobs_last_20_tokens": [(float (np.mean(sequence[(len(sequence)-20):(len(sequence))]))) for sequence in logprob_sum],
        "top_logprobs_first_coding_token":coding_log_probs["first_token"],
        "top_logprobs_first_20_coding_tokens":coding_log_probs["first_20_token"],
        "top_logprobs_last_coding_tokens": coding_log_probs["last_token"],
        "top_logprobs_last_20_coding_tokens":coding_log_probs["last_20_token"],
        "difficulty": sample["difficulty"],
    }


In [26]:

def is_rank(prob_entry:dict, rank:Optional[int]=1) -> bool:
    return prob_entry['rank']==rank

def extract_code_tokens(probs_list:List[dict]) -> List[dict]:
    """Extract tokens and probabilities for the last ```python ... ``` block.
        param probs_list: list of token probs for 1 generation
    """
    
    top_tokens = [x for entry in probs_list for x in filter(is_rank, entry.values())]
    tokens = [t['decoded_token'] for t in top_tokens]
    
    # Join tokens into a single string to find indices
    joined = "".join(tokens)
    start_marker = "```python"
    end_marker = "```"

    # Find last start and end indices in the joined string
    start_idx = joined.rfind(start_marker)
    if start_idx == -1:
        return [] # None, None, []

    end_idx = joined.find(end_marker, start_idx + len(start_marker))
    if end_idx == -1:
        return [] # start_idx, None, []

    # Map character indices back to token indices
    cumulative = 0
    start_token, end_token = None, None
    for i, tok in enumerate(tokens):
        if start_token is None and cumulative >= start_idx:
            start_token = i
        if cumulative >= end_idx:
            end_token = i
            break
        cumulative += len(tok)

    # Slice token/probability lists
    if start_token is not None and end_token is not None:
        return [start_token, end_token] #, top_tokens[start_token:end_token]
    else:
        return [] #None, None, []
    


In [ ]:
def preprocess_data(input_path, output_path, max_samples=None): 
    i = 0
    samples_without_topics = []
    topic_list = []
    with open(input_path, "rb") as infile, open(output_path, "wb") as outfile:
        for line in tqdm(infile):
            if max_samples and i>=max_samples:
                break
            obj = orjson.loads(line)
            
            
            sample = prepare_sample(obj)
            if len(sample["topics"])<1:
                samples_without_topics.append(sample["id"])
            for topic in sample["topics"]:
                if topic not in topic_list:
                    topic_list.append(topic)

            line = orjson.dumps(sample) + b"\n"
            outfile.write(line)
            outfile.write(line)
            print(line)
            i += 1
    #print(topic_list)
    #print(samples_without_topics)
    #print(len(samples_without_topics))

preprocess_data(jsonlines, evaluation_data_path, max_samples=10)

468it [1:02:17, 104.25s/it]

abc303_d
Shift vs. CapsLock
Your computer has a keyboard with three keys: 'a' key, Shift key, and Caps Lock key.  The Caps Lock key has a light on it.
Initially, the light on the Caps Lock key is off, and the screen shows an empty string.
You can do the following three actions any number of times in any order:

- Spend X milliseconds to press only the 'a' key.  If the light on the Caps Lock key is off, a is appended to the string on the screen; if it is on, A is.
- Spend Y milliseconds to press the 'a' key and Shift key simultaneously.  If the light on the Caps Lock key is off, A is appended to the string on the screen; if it is on, a is.
- Spend Z milliseconds to press the Caps Lock key.  If the light on the Caps Lock key is off, it turns on; if it is on, it turns off.

Given a string S consisting of A and a, determine at least how many milliseconds you need to spend to make the string shown on the screen equal to S.

Input

The input is given from Standard Input in the following form

471it [1:21:44, 207.25s/it]

abc303_e
A Gift From the Stars
A graph with (k+1) vertices and k edges is called a level-k\ (k\geq 2) star if and only if:

- it has a vertex that is connected to each of the other k vertices with an edge, and there are no other edges.

At first, Takahashi had a graph consisting of stars.  He repeated the following operation until every pair of vertices in the graph was connected:

- choose two vertices in the graph.  Here, the vertices must be disconnected, and their degrees must be both 1.  Add an edge that connects the chosen two vertices.

He then arbitrarily assigned an integer from 1 through N to each of the vertices in the graph after the procedure.  The resulting graph is a tree; we call it T.  T has (N-1) edges, the i-th of which connects u_i and v_i.
Takahashi has now forgotten the number and levels of the stars that he initially had.  Find them, given T.

Input

The input is given from Standard Input in the following format:
N
u_1 v_1
\vdots
u_{N-1} v_{N-1}

Output

Suppose 

473it [1:21:45, 101.82s/it]

...

abc351_c
Merge the balls
You have an empty sequence and N balls. The size of the i-th ball (1 \leq i \leq N) is 2^{A_i}.
You will perform N operations.
In the i-th operation, you add the i-th ball to the right end of the sequence, and repeat the following steps:

- If the sequence has one or fewer balls, end the operation.
- If the rightmost ball and the second rightmost ball in the sequence have different sizes, end the operation.
- If the rightmost ball and the second rightmost ball in the sequence have the same size, remove these two balls and add a new ball to the right end of the sequence with a size equal to the sum of the sizes of the two removed balls. Then, go back to step 1 and repeat the process.

Determine the number of balls remaining in the sequence after the N operations.

Input

The input is given from Standard Input in the following format:
N
A_1 A_2 \ldots A_N

Output

Print the number of balls in the sequence after the N operations.

Constraints


- 1 \leq N \leq 2 \

703it [1:24:09,  1.17it/s]

abc352_c
Standing On The Shoulders
There are N giants, named 1 to N. When giant i stands on the ground, their shoulder height is A_i, and their head height is B_i.
You can choose a permutation (P_1, P_2, \ldots, P_N) of (1, 2, \ldots, N) and stack the N giants according to the following rules:

- 
First, place giant P_1 on the ground. The giant P_1's shoulder will be at a height of A_{P_1} from the ground, and their head will be at a height of B_{P_1} from the ground.

- 
For i = 1, 2, \ldots, N - 1 in order, place giant P_{i + 1} on the shoulders of giant P_i. If giant P_i's shoulders are at a height of t from the ground, then giant P_{i + 1}'s shoulders will be at a height of t + A_{P_{i + 1}} from the ground, and their head will be at a height of t + B_{P_{i + 1}} from the ground.


Find the maximum possible height of the head of the topmost giant P_N from the ground.

Input

The input is given from Standard Input in the following format:
N
A_1 B_1
A_2 B_2
\vdots
A_N B_N

Output

Pr

703it [1:24:10,  7.18s/it]


KeyboardInterrupt: 

In [21]:
# Full preprocessing pipeline for LiveCodeBench

# Qwen3-Coder
in_path_qwen3 = "output/Qwen3-Coder-30B-A3B/Scenario.codegeneration_10_0.2_eval_all.json"
jsonl_path_qwen3 = "output/Qwen3-Coder-30B-A3B/Scenario.codegeneration_10_0.2_eval_all.jsonl"
out_path_qwen3 = "output/Qwen3-Coder-30B-A3B/preprocessed/codegeneration_10_0.2.jsonl"

# gpt-oss
in_path_gpt_oss = "output/GPT-OSS-20B/Scenario.codegeneration_10_0.2_eval_all.json"
jsonl_path_gpt_oss = "output/GPT-OSS-20B/codegeneration_10_0.2_eval_all.jsonl"
out_path_gpt_oss = "output/GPT-OSS-20B/preprocessed/codegeneration_10_0.2.jsonl"


def prepare_lcb_for_evaluation(in_path, jsonl_path, out_path):
    convert_to_jsonl(in_path, jsonl_path)
    preprocess_data(jsonl_path, out_path)
    
prepare_lcb_for_evaluation(in_path=in_path_gpt_oss,
                           jsonl_path=jsonl_path_gpt_oss,
                           out_path=out_path_gpt_oss)

# prepare_lcb_for_evaluation(in_path=in_path_qwen3,
#                            jsonl_path=jsonl_path_qwen3,
#                            out_path=out_path_qwen3)


FileNotFoundError: [Errno 2] No such file or directory: 'output/GPT-OSS-20B/Scenario.codegeneration_10_0.2_eval_all.json'

### Groups
- loc
- cyclomatic complexity
- len output
- len code
- difficulty
- len prompt
- language

(identifier count, etc)

### Metrics
- mean logprob
@ Katharina:
- sum top-20 logprobs (last n tokens, last token, last code token, first token)
     1) last 20 tokens (Konfigurierbar für n)
     2) last token
     3) first 20 tokens
     4) last 20 code tokens
     5) last code token
     6) first code token